In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import torch
import torch.nn as nn

#approach: utilize a model for each output coordinate
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import RepeatedKFold, cross_val_score

In [2]:
training_path='/kaggle/input/stanford-rna-3d-folding/train_sequences.csv'
validation_path='/kaggle/input/stanford-rna-3d-folding/validation_sequences.csv'
testing_path='/kaggle/input/stanford-rna-3d-folding/test_sequences.csv'

training_labels_path='/kaggle/input/stanford-rna-3d-folding/train_labels.csv'
validation_labels_path='/kaggle/input/stanford-rna-3d-folding/validation_labels.csv'

submission_path='/kaggle/input/stanford-rna-3d-folding/sample_submission.csv'

In [3]:
train_df=pd.read_csv(training_path)
update_train_df=train_df[train_df['temporal_cutoff']<"2022-05-27"]
update_train_df.tail(1)

,target_id,sequence,temporal_cutoff,description,all_sequences
691,7S3H_R,GGCUGCGUAUUUCUACUCUGUUGUUUUAGAGCUAGAAAUAGCAAGU...,2022-04-20,Cas9:sgRNA:DNA (S. pyogenes) with 0 RNA:DNA ba...,>7S3H_1|Chain A[auth N]|Non-target DNA strand|...


In [4]:
train_labels_df=pd.read_csv(training_labels_path)
train_labels_df.head(1)

,ID,resname,resid,x_1,y_1,z_1
0,1SCL_A_1,G,1,13.76,-25.974001,0.102


**Sequence structure prediction preprocessing**

In [5]:
#Extract target label: Protein Data Bank id, Monomer chain id
train_labels_df['target_id']=train_labels_df['ID'].str.rsplit('_',n=1).str[0]
train_labels_df['pdb_monomer']=train_labels_df['ID'].str.rsplit('_',n=1).str[1]

#update submission df
submission_df=pd.read_csv(submission_path)
submission_df['target_id']=submission_df['ID'].str.rsplit('_',n=1).str[0]
submission_df['pdb_monomer']=submission_df['ID'].str.rsplit('_',n=1).str[1]

train_full_df=update_train_df.merge(train_labels_df, how="left", on="target_id")

#train_labels_df.head(1)
train_full_df.head(1)
#submission_df.head(1)

,target_id,sequence,temporal_cutoff,description,all_sequences,ID,resname,resid,x_1,y_1,z_1,pdb_monomer
0,1SCL_A,GGGUGCUCAGUACGAGAGGAACCGCACCC,1995-01-26,"THE SARCIN-RICIN LOOP, A MODULAR RNA",>1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus n...,1SCL_A_1,G,1,13.76,-25.974001,0.102,1


In [6]:
#fill in nan description in all_sequences with "missing"
train_full_df['all_sequences']=train_full_df['all_sequences'].astype(str).fillna("missing")

#fill in missing coordinates with mean values
training_coordinates=['x_1','y_1','z_1']
for t in training_coordinates:
    t_mean=train_full_df[t].mean()
    train_full_df[t]=train_full_df[t].fillna(t_mean)

train_full_df.isna().sum()

target_id          0
sequence           0
temporal_cutoff    0
description        0
all_sequences      0
ID                 0
resname            0
resid              0
x_1                0
y_1                0
z_1                0
pdb_monomer        0
dtype: int64

In [7]:
def df_label_extract_merge(df_path, label_path):
    df=pd.read_csv(df_path)
    ldf=pd.read_csv(label_path)
    ldf['target_id']=ldf['ID'].str.rsplit('_',n=1).str[0]
    ldf['pdb_monomer']=ldf['ID'].str.rsplit('_',n=1).str[1]
    return df.merge(ldf, how='left', on='target_id')

In [8]:
#validation has 40 structural coordinate output triplets, whereas training has only 1 triplet output
validation_df=df_label_extract_merge(df_path=validation_path, label_path=validation_labels_path)
validation_df.head(1)

,target_id,sequence,temporal_cutoff,description,all_sequences,ID,resname,resid,x_1,y_1,...,x_38,y_38,z_38,x_39,y_39,z_39,x_40,y_40,z_40,pdb_monomer
0,R1107,GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUU...,2022-05-28,CPEB3 ribozyme\nHuman\nhuman CPEB3 HDV-like ri...,>7QR4_1|Chain A|U1 small nuclear ribonucleopro...,R1107_1,G,1,-5.499,8.52,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,1


In [9]:
#convert submission csv as same format as original training csv
test_df=pd.read_csv(testing_path)
test_complete_df=test_df.merge(submission_df, how="left", on="target_id")

In [10]:
print(train_full_df.loc[6500,'all_sequences'])

>1ZC8_1|Chain A|TLD 16S ribosomal RNA|Thermus thermophilus (274)
GGGGCUGAUUCUGGAUUCGACGGGAUAUUUCGGACGCGGGUUCAACUCCCGCCAGCUCC
>1ZC8_10|Chain J[auth K]|SsrA-binding protein|Thermus thermophilus (274)
GKSDKIIPIAENKEAKAKYDILETYEAGIVLKGSEVKSLREKGTVSFKDSFVRIENGEAWLYNLYIAPYKHATIENHDPLRKRKLLLHKREIMRLYGKVQEKGYTIIPLKLYWKNNKVKVLIALAKGKKL
>1ZC8_11|Chain K[auth Y]|Elongation factor Tu|Thermus thermophilus (274)
AKGEFIRTKPHVNVGTIGHVDHGKTTLTAALTYVAAAENPNVEVKDYGDIDKAPEERARGITINTAHVEYETAKRHYSHVDCPGHADYIKNMITGAAQMDGAILVVSAADGPMPQTREHILLARQVGVPYIVVFMNKVDMVDDPELLDLVEMEVRDLLNQYEFPGDEVPVIRGSALLALEEMHKNPKTKRGENEWVDKIWELLDAIDEYIPTPVRDVDKPFLMPVEDVFTITGRGTVATGRIERGKVKVGDEVEIVGLAPETRKTVVTGVEMHRKTLQEGIAGDNVGLLLRGVSREEVERGQVLAKPGSITPHTKFEASVYILKKEEGGRHTGFFTGYRPQFYFRTTDVTGVVRLPQGVEMVMPGDNVTFTVELIKPVALEEGLRFAIREGGRTVGAGVVTKILE
>1ZC8_2|Chain B|H2 16S rRNA|Thermus thermophilus (274)
UUGCGAAACAUGUAGG
>1ZC8_3|Chain C|H2b d mRNA|Thermus thermophilus (274)
CCCAAGGUGCAUGCGCAUGUAGUACCGAGGA
>1ZC8_4|Chain D[auth G]|protein ki

**Feature Engineering**
- Add 3D structural information in the PDB database https://search.rcsb.org/#search-services
- data augmentation & transfer learning: SPOT-RNA, pretrained on bpRNA database (stems, hairpin loops and pseudoknots)
- sequences converted to 2D images: MXFold2 utilizes UFold process to then input into a CNN and efficiently generate results
- ARES atom-level information instead of nucleotides (small training set used)
- EternalFold: uses chemical mapping, and riboswitch–ligand binding affinity data


In [11]:
print(torch.cuda.is_available())
len(train_full_df['pdb_monomer'].unique())

True


4298

In [12]:
#simple sequence length feature
train_full_df['seq_len']=train_full_df['sequence'].str.len()
test_complete_df['seq_len']=test_complete_df['sequence'].str.len()

#categorical columns
cat_cols=['target_id', 'resname']

#dataset inspection
print('unique ids:', len(train_full_df['target_id'].unique()))
print('unique resname:', len(train_full_df['resname'].unique()))
print(train_full_df['resname'].unique())
print('total datapoints:', len(train_full_df))

#Encode and create an embedding features from the target ids
encoded_ids, id_maps= pd.factorize(train_full_df['target_id'])
id_tensor=torch.tensor(encoded_ids, dtype=torch.long)
embed_size=int(len(train_full_df['target_id'].unique()))
embedding_dim=min(50, np.power(len(train_full_df['target_id'].unique()),0.25))
#print(np.ceil(embedding_dim))
embed_dim=int(np.ceil(embedding_dim))
embed_layer=nn.Embedding(num_embeddings=embed_size, embedding_dim=embed_dim)
embed_ids=embed_layer(id_tensor).detach().cpu().numpy()
embed_id_df=pd.DataFrame(embed_ids, columns=[f'embedim_{i}' for i in range(embed_dim)])

#TF-IDF transform description and all-sequences columns into word embedding features
tfidf_vec=TfidfVectorizer(min_df=5, max_df=0.9, max_features=5000) 

train_full_df['combined_descrip_allseqs']=train_full_df['description']+' '+train_full_df['all_sequences']

tfidf_all=tfidf_vec.fit_transform(train_full_df['combined_descrip_allseqs'])

#after inspecting some allsequence text some have 'xxxxxxxxxx'
# ft_names=list(tfidf_vec.get_feature_names_out())

# pattern = f"({re.escape('x')})\\1*"
# filter_words=[f for f in ft_names if not f.isnumeric() and not re.fullmatch(pattern, f)]

# print(len(filter_words))

tfidf_all_df=pd.DataFrame(tfidf_all.toarray(), columns=tfidf_vec.get_feature_names_out())

tfidf_all_df.head(3)

unique ids: 692
unique resname: 6
['G' 'U' 'C' 'A' '-' 'X']
total datapoints: 113616


,01,02,03,04,05,10,100440,10045,100673,10090,...,zb,zc,zd,ze,zf,zg,zh,zi,zika,zz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#try to reduce the number of word features
var_selector=VarianceThreshold(threshold=0.01)
da_select=var_selector.fit_transform(tfidf_all_df) #transform on test df
ft_names=var_selector.get_feature_names_out()
non_num_ftnames=[f for f in ft_names if not f.isnumeric()]
reduced_wfts_df=pd.DataFrame(da_select, columns=ft_names)[non_num_ftnames]
reduced_wfts_df.head(3)

,aestivum,auth,bacillus,coli,cuniculus,drosophila,escherichia,homo,jannaschii,melanogaster,methanococcus,oryctolagus,sapiens,subtilis,thermophilus,thermus,triticum
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#Encode the resname
ohe_resname=pd.get_dummies(train_full_df['resname'], dtype='int')
#ohe_resname
# ohe_rdf=pd.DataFrame(ohe_resname.toarray(), columns=ohe.get_features_names_out())
# ohe_rdf.head(3)

In [15]:
train_full_df['pdb_monomer']=train_full_df['pdb_monomer'].astype(int)
train_full_df['pdb_monomer'].unique()

array([   1,    2,    3, ..., 4296, 4297, 4298])

In [16]:
#numerical columns
num_cols=['resid', 'pdb_monomer', 'seq_len']

#TODO: merge features into X and run model; catBoost can use train_full_df['sequence']
full_prep_train=pd.concat([train_full_df[num_cols], reduced_wfts_df, ohe_resname], axis=1)
full_prep_train.head(3)

,resid,pdb_monomer,seq_len,aestivum,auth,bacillus,coli,cuniculus,drosophila,escherichia,...,subtilis,thermophilus,thermus,triticum,-,A,C,G,U,X
0,1,1,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
1,2,2,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
2,3,3,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0


In [17]:
#y: coordinates
y_list=training_coordinates

model_yi={}

for y in y_list:
    xgboost_model=xgb.XGBRegressor(
        objective='reg:squarederror', #since there are negative coordinates
        n_estimators=100 , 
        max_depth=7,
        learning_rate=0.1,
        subsample=0.6,
        colsample_bytree=0.6
        )
    
    xgboost_model.fit(full_prep_train, train_full_df[y])
    model_yi[y]=xgboost_model

In [18]:
model_yi['x_1']

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)